In [1]:
from drn_interactions.io import load_events, load_derived_generic
from drn_interactions.transforms.spikes import SpikesHandlerMulti
from drn_interactions.responders.fs_slow import (
    SlowTSRespondersMixed, SlowTSRespondersAnova, plot_pop, population_raster
)

import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import numpy as np
import pandas as pd
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [ ]:
# sns.set_theme(context="poster", style="ticks")
# 

In [2]:

outdir = get_fig_dir() / "base_shock"
outdir.mkdir(exist_ok=True, parents=True)

neurons = load_neurons_derived()
clusters = neurons[["neuron_id", "wf_3"]]
events = load_events("base_shock")
sessions = neurons[["session_name"]].drop_duplicates().merge(events[["session_name"]].drop_duplicates())["session_name"].unique()
# [plot_pop(session, z=True) for session in sessions]

In [ ]:
t_start=-600
t_stop=1200
bin_width=1
session = sessions


sh = SpikesHandlerMulti(
    block=["base_shock", "post_base_shock"], 
    bin_width=bin_width, 
    session_names=sessions.tolist(),
    t_start=t_start, 
    t_stop=t_stop,
)
ax = sh.binned_piv.sum(axis=1).plot(color="black")
ax.set_title("All")
sns.despine()
df_binned_piv = sh.binned_piv.copy()

In [ ]:
# Reapeated measures

# Using Clusters
mod = SlowTSRespondersAnova()
mod.get_responders(df_binned_piv=df_binned_piv, clusters=neurons[["neuron_id", "wf_3"]],)
display(mod.results_["anova"])
display(mod.results_["contrasts"])
print()

# Without using clusters
mod = SlowTSRespondersAnova()
mod.get_responders(df_binned_piv=df_binned_piv, fit_unit_level_models=True)
display(mod.results_["anova"])
display(mod.results_["contrasts"])



In [ ]:
# Plot Single Units
f = mod.plot_unit_effects()
f.show()

In [ ]:
# Plot Population Raster
population_raster(sessions[4], tfidf=True)

In [ ]:
# mixed effects alternative method

# Using Clusters
mod = SlowTSRespondersMixed()
mod.get_responders(df_binned_piv, clusters = neurons[["neuron_id","wf_3"]])
display(mod.results_["anova"])


# Without Using Clusters

mod = SlowTSRespondersMixed()
mod.get_responders(df_binned_piv)
display(mod.results_["anova"])
display(mod.results_["emms"])
display(mod.results_["contrasts"])
# mod.plot()